### Using LSA to train a classifier for character gender recognition in Shakespeare plays.

In [27]:
import os
import os.path
import shutil
import numpy as np
import pandas as pd
import random
import pickle

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.externals import joblib
from sklearn.externals.joblib import Memory

import nltk
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
%matplotlib inline

We first need to download punkt, our sentence tokenizer, and also a dictionary of stopwords that we'll use later to remove stopwords from the texts.

In [5]:
%%capture
nltk.download('punkt')
nltk.download('stopwords')

We'll also need to find and assign genders to all characters and their associated texts. We'll use [this list of female characters](http://www.shakespeareswords.com/Special-Features-Female-Characters).

In [6]:
female_characters_df = pd.read_csv("female_characters.csv")
female_characters_df.character = female_characters_df.character.str.lower()

In [7]:
female_characters_df.sample(n= 3)

,character,play_name,gender,replics
139,lychorida,"Pericles, Prince of Tyre",Female,11
121,gentlewoman,Macbeth,Female,24
155,timandra,Timon of Athens,Female,4


Now we'll de-serialize the data streams from our pickle file, which contain all texts from all of Shakespeare's plays along with the associated play, speaker, act/scene, and play genre.

In [8]:
with open('shakespeare_plays.pickle', 'rb') as handle:
    speeches = pickle.load(handle)

In [9]:
speeches_df = pd.DataFrame(speeches)
speeches_df.speaker = speeches_df.speaker.str.lower()

In [10]:
speeches_df.sample(n = 3)

,act,genre,play_name,scene,scene_name,speaker,speech_number,speech_text
14217,2,History,Henry V,1,London. A street.,boy,24,"Mine host Pistol, you must come to my master, ..."
9206,2,Comedy,Taming of the Shrew,1,Padua. A room in BAPTISTA'S house.,hortensio,41,"For fear, I promise you, if I look pale."
4875,4,Comedy,Measure for Measure,2,A room in the prison.,provost,31,It is a bitter deputy.


We'll also double check that our list of plays are the same as the plays listed in the genders dataframe.

In [11]:
our_names = set(speeches_df.play_name)
print(our_names)

{'Two Gentlemen of Verona', 'The Tempest', 'Pericles, Prince of Tyre', 'Cymbeline', 'Twelfth Night', 'Timon of Athens', 'The Merchant of Venice', 'Julius Caesar', 'Henry VIII', "A Midsummer Night's Dream", 'Macbeth', "All's Well That Ends Well", "Love's Labours Lost", 'Much Ado About Nothing', 'Taming of the Shrew', 'TheMerry Wives of Windsor', 'King John', 'The Comedy of Errors', 'Othello', 'As You Like It', 'Richard II', 'King Lear', 'Troilus and Cressida', 'Richard III', 'Coriolanus', 'Hamlet', 'Measure for Measure', "Winter's Tale", 'Antony and Cleopatra', 'Henry V', 'Romeo and Juliet', 'Titus Andronicus'}


In [12]:
their_names = set(female_characters_df.play_name.unique())

In [13]:
print(their_names)

{'Two Gentlemen of Verona', 'The Tempest', 'Pericles, Prince of Tyre', 'Cymbeline', 'Twelfth Night', 'Timon of Athens', 'The Merchant of Venice', 'Julius Caesar', 'Henry VIII', 'Macbeth', "A Midsummer Night's Dream", "All's Well That Ends Well", "Love's Labours Lost", 'Much Ado About Nothing', 'Taming of the Shrew', 'TheMerry Wives of Windsor', 'Henry IV', 'King John', 'The Comedy of Errors', 'Othello', 'As You Like It', 'Richard II', 'King Lear', 'Troilus and Cressida', 'Coriolanus', 'Richard III', 'Hamlet', 'Henry VI', 'Measure for Measure', "Winter's Tale", 'Antony and Cleopatra', 'Romeo and Juliet', 'Titus Andronicus', 'Henry V', 'The Two Noble Kinsmen', 'King Edward III'}


Now we check to see if our texts include any plays that aren't included in the genders dataframe and note that there are none.

In [14]:
print(our_names - their_names)

set()


On the other hand, it looks like the additional plays in the genders dataframe are those that Shakespeare co-authored, so looks we're looking okay.

In [15]:
print(their_names - our_names)

{'King Edward III', 'Henry IV', 'Henry VI', 'The Two Noble Kinsmen'}


Now we create sets of our speakers and add a 'female' column to the dataset.

In [16]:
our_speakers = set(speeches_df.speaker)
their_speakers = set(female_characters_df.character)

speeches_df['female'] = speeches_df.apply(lambda r : r['speaker'] in their_speakers, axis=1)

We also note that some of the texts are spoken by multiple people:

In [17]:
speeches_df[speeches_df.speaker.str.startswith('all')][0:3]

,act,genre,play_name,scene,scene_name,speaker,speech_number,speech_text,female
268,2,Comedy,All's Well That Ends Well,3,Paris. The KING's palace.,all,34,"We understand it, and thank heaven for you.",False
587,4,Comedy,All's Well That Ends Well,1,Without the Florentine camp.,all,26,"Cargo, cargo, cargo, villiando par corbo, cargo.",False
3062,5,Comedy,Cymbeline,4,A British prison.,all,19,"Thanks, Jupiter!",False


Because we are examining differences in male/female speech in this analysis, we will discard all text spoken by multiple people.

In [18]:
speeches_df.drop(speeches_df[speeches_df.speaker.str.startswith('all')].index, inplace = True)
speeches_df.sample(n = 3)

,act,genre,play_name,scene,scene_name,speaker,speech_number,speech_text,female
26939,4,Tragedy,Titus Andronicus,3,The same. A public place.,clown,18,"Ay, of my pigeons, sir; nothing else.",False
13974,4,Comedy,Winter's Tale,4,The Shepherd's cottage.,autolycus,210,What advocate hast thou to him?,False
10958,3,Comedy,Troilus and Cressida,1,Troy. Priam's palace.,paris,55,I spy.,False


Now that we have a dataframe with all the data that we need, we turn our attention to the imbalance in male/female texts. We will have to take this into account when fitting the model.

In [19]:
speeches_df.groupby(['female']).size()

female
False    22275
True      4728
dtype: int64

In [20]:
# 0 - male, 1 - female
labels = [ 1 if f else 0 for f in speeches_df.female.values ]

Now we can perform text preprocessing, steming and tokenizing our words.

This will not used in the final version as it makes CountVectorizer very slow. The built in functionality will be used instead.

In [21]:
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

Below we note that we've successfully tokenized and stemmed our words.'

In [22]:
t = speeches_df.loc[334, 'speech_text']
print(t)

Although before the solemn priest I have sworn,
I will not bed her.


In [23]:
print(tokenize(t))

['although', 'befor', 'the', 'solemn', 'priest', 'I', 'have', 'sworn', ',', 'I', 'will', 'not', 'bed', 'her', '.']


Finally, we can begin the analysis, splitting our features into test and train sets.

In [24]:
features = speeches_df['speech_text'].values

In [25]:
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels, 
                                                    stratify = labels, 
                                                    test_size = 0.10, 
                                                    random_state = 100)

Now we can create a feature space, perform dimension reduction, and search for the best preforming parameters. We'll try three models: an SGDClassifier, RandomForestClassifier, and KNeighborsClassifier.

In [63]:
shutil.rmtree('pipeline_sgd', ignore_errors = True)
os.makedirs('pipeline_sgd')

pipe_sgd = Pipeline(
    memory = 'pipeline',
    steps=[
        # Create feature space
        ('vect', CountVectorizer(min_df=2, 
                                 stop_words='english', 
                                 lowercase=True,
                                 strip_accents='unicode')),
                                 #ngram_range=(2,3))
        ('tfidf', TfidfTransformer()),        
        # Perform LSA on the features
        ('svd', TruncatedSVD()),
        # faster than SVC, default loss is 'hinge'
        ('clf', SGDClassifier(class_weight= 'balanced', 
                              verbose = 0, 
                              n_jobs = -1, 
                              max_iter = 1000))
    ]
)

param_grid_sgd = {
    'vect__ngram_range': ((1, 1), (1, 2)), # unigrams or bigrams
    'tfidf__norm': ['l1', 'l2'],
    'svd__n_components': [250, 300, 350],
    'clf__alpha': [0.00001, 0.000001],
    'clf__penalty': ('l2', 'elasticnet')
}

model_sgd = GridSearchCV(
    pipe_sgd,
    param_grid = param_grid_sgd,
    cv = StratifiedKFold(random_state = 100),
    scoring = 'f1',
    verbose = 1,
    n_jobs = -1)

In [65]:
shutil.rmtree('pipeline_rf', ignore_errors = True)
os.makedirs('pipeline_rf')

pipe_rf = Pipeline(
    memory = 'pipeline',
    steps=[
        # Create feature space
        ('vect', CountVectorizer(min_df=2, 
                                 stop_words='english', 
                                 lowercase=True,
                                 strip_accents='unicode')),
                                 #ngram_range=(2,3))
        ('tfidf', TfidfTransformer()),        
        # Perform LSA on the features
        ('svd', TruncatedSVD()),
        ('rf', RandomForestClassifier(criterion='gini',
                                       min_samples_split=2, 
                                       min_samples_leaf=1, 
                                       min_weight_fraction_leaf=0.0, 
                                       max_features='auto', 
                                       max_leaf_nodes=None, 
                                       min_impurity_decrease=0.0, 
                                       min_impurity_split=None, 
                                       bootstrap=True, 
                                       oob_score=False, 
                                       n_jobs=1, 
                                       random_state=None, 
                                       verbose=0, 
                                       warm_start=False, 
                                       class_weight=None))
    ]
)

param_grid_rf = {
    'vect__ngram_range': ((1, 1), (1, 2)), # unigrams or bigrams
    'tfidf__norm': ['l1', 'l2'],
    'svd__n_components': [250, 300, 350],
    'rf__n_estimators':[5,10,15]
}

model_rf = GridSearchCV(
    pipe_rf,
    param_grid = param_grid_rf,
    cv = StratifiedKFold(random_state = 100),
    scoring = 'f1',
    verbose = 1,
    n_jobs = -1)

In [64]:
shutil.rmtree('pipeline_knn', ignore_errors = True)
os.makedirs('pipeline_knn')

pipe_knn = Pipeline(
    memory = 'pipeline',
    steps=[
        # Create feature space
        ('vect', CountVectorizer(min_df=2, 
                                 stop_words='english', 
                                 lowercase=True,
                                 strip_accents='unicode')),
                                 #ngram_range=(2,3))
        ('tfidf', TfidfTransformer()),        
        # Perform LSA on the features
        ('svd', TruncatedSVD()),
        ('knn',KNeighborsClassifier(weights='uniform', 
                                    algorithm='auto', 
                                    p=2, 
                                    metric='minkowski', 
                                    metric_params=None, 
                                    n_jobs=1))
    ]
)

param_grid_knn = {
    'vect__ngram_range': ((1, 1), (1, 2)), # unigrams or bigrams
    'tfidf__norm': ['l1', 'l2'],
    'svd__n_components': [250, 300, 350],
    'knn__n_neighbors':[3,4,5,6,7,8]
}

model_knn = GridSearchCV(
    pipe_knn,
    param_grid = param_grid_knn,
    cv = StratifiedKFold(random_state = 100),
    scoring = 'f1',
    verbose = 1,
    n_jobs = -1)

In [66]:
model_sgd = model_sgd.fit(X_train, y_train)
model_rf = model_rf.fit(X_train, y_train)
model_knn = model_knn.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 23.7min finished


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  4.5min finished


Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 50.9min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 246.2min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed: 280.4min finished


We can take a look at how the model's performance varied with different parameters:

In [81]:
results_sgd = pd.DataFrame({'mean_test_score': np.array(model_sgd.cv_results_['mean_test_score']),
                        'tfidf__norm': np.array(model_sgd.cv_results_['param_tfidf__norm']),
                        'svd__n_components': np.array(model_sgd.cv_results_['param_svd__n_components']),
                        'clf__alpha': np.array(model_sgd.cv_results_['param_clf__alpha']),
                        'clf__penalty': np.array(model_sgd.cv_results_['param_clf__penalty'])})

results_rf = pd.DataFrame({'mean_test_score': np.array(model_rf.cv_results_['mean_test_score']),
                        'tfidf__norm': np.array(model_rf.cv_results_['param_tfidf__norm']),
                        'svd__n_components': np.array(model_rf.cv_results_['param_svd__n_components']),
                        'rf__n_estimators': np.array(model_rf.cv_results_['param_rf__n_estimators'])})

results_knn = pd.DataFrame({'mean_test_score': np.array(model_knn.cv_results_['mean_test_score']),
                        'tfidf__norm': np.array(model_knn.cv_results_['param_tfidf__norm']),
                        'svd__n_components': np.array(model_knn.cv_results_['param_svd__n_components']),
                        'knn__n_neighbors': np.array(model_knn.cv_results_['param_knn__n_neighbors'])})

print(results_sgd.head())
print(results_rf.head())
print(results_knn.head())

  clf__alpha clf__penalty  mean_test_score svd__n_components tfidf__norm
0      1e-05           l2         0.283419               250          l1
1      1e-05           l2         0.251819               250          l1
2      1e-05           l2         0.321952               250          l2
3      1e-05           l2         0.325283               250          l2
4      1e-05           l2         0.289454               300          l1
   mean_test_score rf__n_estimators svd__n_components tfidf__norm
0         0.131355                5               250          l1
1         0.145271                5               250          l1
2         0.141113                5               250          l2
3         0.138767                5               250          l2
4         0.131378                5               300          l1
  knn__n_neighbors  mean_test_score svd__n_components tfidf__norm
0                3         0.152070               250          l1
1                3         0.15225

And we can see the best set of parameters for each of the models:

In [87]:
print("The best parameters are %s with a score of %0.2f" % (model_sgd.best_params_, model_sgd.best_score_))
print()
print("The best parameters are %s with a score of %0.2f" % (model_rf.best_params_, model_rf.best_score_))
print()
print("The best parameters are %s with a score of %0.2f" % (model_knn.best_params_, model_knn.best_score_))

The best parameters are {'clf__alpha': 1e-05, 'clf__penalty': 'elasticnet', 'svd__n_components': 350, 'tfidf__norm': 'l2', 'vect__ngram_range': (1, 1)} with a score of 0.34

The best parameters are {'rf__n_estimators': 5, 'svd__n_components': 250, 'tfidf__norm': 'l1', 'vect__ngram_range': (1, 2)} with a score of 0.15

The best parameters are {'knn__n_neighbors': 3, 'svd__n_components': 350, 'tfidf__norm': 'l2', 'vect__ngram_range': (1, 2)} with a score of 0.16


And finally we can test on our test data and compare the results of the best set of parameters for each of our models.

In [83]:
y_hat_sgd = model_sgd.predict(X_test)
y_hat_rf = model_rf.predict(X_test)
y_hat_knn = model_knn.predict(X_test)

In [84]:
print(classification_report(y_test, y_hat_sgd))

             precision    recall  f1-score   support

          0       0.86      0.65      0.74      2228
          1       0.23      0.49      0.31       473

avg / total       0.75      0.62      0.66      2701



In [85]:
print(classification_report(y_test, y_hat_rf))

             precision    recall  f1-score   support

          0       0.83      0.93      0.88      2228
          1       0.22      0.09      0.13       473

avg / total       0.72      0.78      0.75      2701



In [86]:
print(classification_report(y_test, y_hat_knn))

             precision    recall  f1-score   support

          0       0.83      0.93      0.87      2228
          1       0.19      0.08      0.12       473

avg / total       0.72      0.78      0.74      2701



We can compare this against using only the training set class distribution:

In [28]:
dummy = Pipeline(
    steps=[
        ('vect', CountVectorizer()), 
        ('clf', DummyClassifier(strategy='stratified', random_state=100))
    ]
)
dummy = dummy.fit(X_train, y_train)
y_dummy = dummy.predict(X_test)

In [29]:
print('Classification report for the dummy classifier')
print(classification_report(y_test, y_dummy))

Classification report for the dummy classifier
             precision    recall  f1-score   support

          0       0.83      0.84      0.83      2228
          1       0.18      0.16      0.17       473

avg / total       0.71      0.72      0.72      2701



Comparing the dummy classifier against our own,it looks like our classifiers can do only slightly better in recognizing female characters. Overall we think we've failed to achive a significant improvement over the dummy classifier. 

We have to conclude that it is not possible to recoginze the characters' gender just by one speech. Additionally, we think it is a hard task and better results could be achieved if we used a combination of all speeches per character as the observations.